In [16]:
from my_config import * 

import os

In [ ]:
torch.cuda.is_available()

In [18]:
import import_ipynb
import data_processing
import custom_model

In [19]:
indices = list(range(100))
subset = Subset(data_processing.celeba_dataset,indices)

In [20]:
subset_loader = DataLoader(subset,batch_size=10,shuffle=True)

In [21]:
mymodel = custom_model.model


In [22]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mymodel.parameters(), lr=0.001, weight_decay=1e-5)


In [23]:
checkpoint_path = './checkpoint.pth'

In [24]:
# 保存模型和优化器的状态
def save_checkpoint(model, optimizer, epoch, loss, checkpoint_path=checkpoint_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, checkpoint_path)
    print(f"Model saved at epoch {epoch}, with loss: {loss:.4f}")


In [25]:
def load_checkpoint(model, optimizer, checkpoint_path=checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Loaded model from epoch {epoch} with loss: {loss:.4f}")
    return epoch, loss

In [ ]:
for images, labels in subset_loader:
    print(f'Input images shape: {images.shape}, Labels shape: {labels.shape}')
    break  # 只检查第一批

In [27]:
def train_model(mymodel, train_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda")
    mymodel.to(device)  # 移动模型到 GPU
    if os.path.isfile(checkpoint_path):
        start_epoch, previous_loss = load_checkpoint(mymodel, optimizer, checkpoint_path=checkpoint_path)
    else:
        print("Starting training from scratch.")
        start_epoch = 0  # 如果没有加载检查点，则从0开始
        previous_loss = float('inf')
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    
        
    for epoch in range(start_epoch,num_epochs):
        print(f"epoch:{epoch}")
        mymodel.train()
        running_loss = 0.0
        for images, labels in train_loader:
            
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()  # 清空梯度

            age_labels = labels[:, 0].float().view(-1, 1)
            race_labels = labels[:, 1].float().view(-1, 1)
            gender_labels = labels[:, 2].float().view(-1, 1)

            
            # 前向传播
            age_out, race_out, gender_out = mymodel(images)
            print(f"age_out:{age_out}")
            print(f"age_out shape: {age_out.shape}, age_labels shape: {age_labels.shape}")
                # 计算损失
            loss_age = criterion(age_out, age_labels)
            loss_race = criterion(race_out, race_labels)
            loss_gender = criterion(gender_out, gender_labels)
            print(f"loss_age:{loss_age}")
            total_loss = loss_age + loss_race + loss_gender

            total_loss.backward()  # 反向传播
            optimizer.step()  # 更新权重

            running_loss += total_loss.item()
            print(f"running_loss:{running_loss}")
        
        avg_loss = running_loss / len(train_loader)
        print(f"avg_loss:{avg_loss}")
        scheduler.step(avg_loss)  
        save_checkpoint(mymodel, optimizer, epoch+1, running_loss/len(train_loader))
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')

In [31]:
train_model(mymodel=mymodel,train_loader=subset_loader,criterion=criterion,optimizer=optimizer,num_epochs=10)

Starting training from scratch.
epoch:0
age_out:tensor([[0.7477],
        [0.6918],
        [0.7328],
        [0.7010],
        [0.7266],
        [0.6808],
        [0.7330],
        [0.6937],
        [0.6943],
        [0.7294]], device='cuda:0', grad_fn=<SigmoidBackward0>)
age_out shape: torch.Size([10, 1]), age_labels shape: torch.Size([10, 1])
loss_age:0.420623779296875
running_loss:1.428068995475769
age_out:tensor([[0.7830],
        [0.7687],
        [0.7447],
        [0.7879],
        [0.7570],
        [0.7435],
        [0.7650],
        [0.7449],
        [0.6847],
        [0.7902]], device='cuda:0', grad_fn=<SigmoidBackward0>)
age_out shape: torch.Size([10, 1]), age_labels shape: torch.Size([10, 1])
loss_age:0.38567468523979187
running_loss:3.112121820449829
age_out:tensor([[0.8225],
        [0.8612],
        [0.8646],
        [0.8605],
        [0.8630],
        [0.8878],
        [0.8885],
        [0.8642],
        [0.8927],
        [0.8543]], device='cuda:0', grad_fn=<SigmoidBack

KeyboardInterrupt: 